In [25]:
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import re
import numpy as np

def getData(folder, filename): #I'm finally using python properly
    rdr = ascii.get_reader(Reader=ascii.Basic)
    rdr.header.splitter.delimiter = ' '
    rdr.data.splitter.delimiter = ' '
    if folder == 'DES_BLINDnoHOSTZ':
        rdr.header.start_line = 12
        rdr.data.start_line = 13
    else:
        rdr.header.start_line = 14
        rdr.data.start_line = 15
    rdr.data.end_line = None
    t = rdr.read('%s/%s' % (folder, filename))
    FLT = t['FLT']

    gFlux = np.array(t['FLUXCAL'][FLT == 'g'])
    rFlux = np.array(t['FLUXCAL'][FLT == 'r'])
    iFlux = np.array(t['FLUXCAL'][FLT == 'i'])
    zFlux = np.array(t['FLUXCAL'][FLT == 'z'])
    gErr = np.array(t['FLUXCALERR'][FLT == 'g'])
    rErr = np.array(t['FLUXCALERR'][FLT == 'r'])
    iErr = np.array(t['FLUXCALERR'][FLT == 'i'])
    zErr = np.array(t['FLUXCALERR'][FLT == 'z'])
    maxL = max(len(gFlux),len(rFlux), len(iFlux), len(zFlux))

    for i in range(len(FLT)):
        if i%4 == 0:
            if FLT[i] != 'g': 
                np.insert(gFlux,(i//4),0)
                np.insert(gErr,(i//4),0)
        if i%4 == 1:
            if FLT[i] != 'r': 
                np.insert(rFlux,(i//4),0)
                np.insert(rErr,(i//4),0)
        if i%4 == 2:s
            if FLT[i] != 'i': 
                np.insert(iFlux,(i//4),0)
                np.insert(iErr,(i//4),0)
        if i%4 == 3:
            if FLT[i] != 'z': 
                np.insert(zFlux,(i//4),0)
                np.insert(zErr,(i//4),0)
    
    d = np.array([gFlux,rFlux,iFlux,zFlux, gErr,rErr,iErr,zErr])
    ts = np.zeros((50, 8))
    for j in range(8):
        toto = d[j]
        ts[:len(toto),j] = toto
    return ts
    
def getInfo(info, file):
    for line in file:
        if line.startswith('%s' % info):
            return line[(len(info)+2)::] #might have extra spaces in front and it's a string
    return None

folder= 'DES_BLINDnoHOSTZ'

#folder: no vs +
#infoL: info list. eg. 'SNID', 'SNTYPE', 'FILTERS', 'RA', 'DECL', 'FAKE', 'MWEBV', ETC.
#totalNum: total number of supernovae in the foler. Might be improved to automatically going through all files in a folder
if folder == 'DES_BLINDnoHOSTZ':
    colName = 'DES_SN000018.DAT'
    totalNum = 18347
    infoL = ['SURVEY','SNID', 'SNTYPE', 'FILTERS', 'RA', 'DECL', 'FAKE', 'MWEBV', 'REDSHIFT_SPE','NOBS', 'NVAR']
else:
    colName = 'DES_SN000041.DAT'
    totalNum = 18320
    infoL = ['SURVEY','SNID','SNTYPE','FILTERS','RA','DECL','FAKE','MWEBV','REDSHIFT_SPE','HOST_GALAXY_GALID','HOST_GALAXY_PHOTO-Z','NOBS','NVAR']

list = ascii.read('%s/%s.LIST' % (folder, folder), data_start = 0)
table = dict()
for i in infoL:
    table[i] = []
table['FILENAME'] = []
table['DATA'] = []
#table
for i in range(totalNum):
    with open(folder + '/' + list[colName][i]) as f:
        table['FILENAME'] += [list[colName][i]]
        table['DATA'] += [getData(folder, list[colName][i])]
        for j in infoL:
            table[j] += [getInfo(j, f)]
infoList = [table['FILENAME']]
for i in infoL:
    infoList += [table[i]]
infoList += [table['DATA']]

IndexError: index 22 is out of bounds for axis 0 with size 21

In [9]:
output = Table(infoList, names = ['FILENAME']+infoL+['DATA'])

'd = infoList[-1]\nts = zeros((len(d), 50, 8))\nfor i in range(len(d)):\n    for j in range(8):\n        toto = d[i][0][j]\n        ts[i,:len(toto),j] = toto'

In [10]:
output

FILENAME,SURVEY,SNID,SNTYPE,FILTERS,RA,DECL,FAKE,MWEBV,REDSHIFT_SPE,NOBS,NVAR,"DATA [50,8]"
str16,str7,str12,str5,str6,str21,str20,str35,str23,str25,str5,str3,float64
DES_SN000018.DAT,DES,18,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,118,5,0.02457 .. 0.0
DES_SN000050.DAT,DES,50,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,16,5,3.231 .. 0.0
DES_SN000051.DAT,DES,51,1,griz,52.500000 deg,-27.500000 deg,3 (=> BLIND-TEST simulation),0.0097 MW E(B-V),0.5177 +- 0.0050,121,5,2.97 .. 0.0
DES_SN000063.DAT,DES,63,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,115,5,14.9 .. 0.0
DES_SN000064.DAT,DES,64,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,92,5,2.485 .. 0.0
DES_SN000096.DAT,DES,96,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,104,5,19.37 .. 0.0
DES_SN000122.DAT,DES,122,-9,griz,34.500000 deg,-5.500000 deg,3 (=> BLIND-TEST simulation),0.0227 MW E(B-V),-9.0000 +- 9.0000,11,5,23.12 .. 0.0
DES_SN000124.DAT,DES,124,-9,griz,48.000000 deg,0.000000 deg,3 (=> BLIND-TEST simulation),0.0713 MW E(B-V),-9.0000 +- 9.0000,95,5,-8.947 .. 0.0
DES_SN000156.DAT,DES,156,-9,griz,36.750000 deg,-4.500000 deg,3 (=> BLIND-TEST simulation),0.0283 MW E(B-V),-9.0000 +- 9.0000,87,5,1.551 .. 0.0
